In [1]:
pwd

'/mnt'

In [2]:
import os

In [3]:
import re

In [4]:
files = os.listdir("/mnt/standard_mvp/Rx/macros")
sasfiles = [f for f in files if f.endswith('.sas')]

In [5]:
import pandas as pd

In [6]:
data = pd.DataFrame(sasfiles, columns = ["sasfiles"])

In [7]:
data.head()

,sasfiles
0,tc_adpft.sas
1,tc_p1_vitals.sas
2,td_pkpt3p.sas
3,td_ae1.sas
4,tu_align.sas


In [8]:
def read_files(filename):
    try:
        
        
        with open("/mnt/standard_mvp/Rx/macros/" + filename,'r',encoding="UTF-8" ) as f:
            my_lines = f.readlines()
        return my_lines
    except Exception as e: 
        print(filename)
        print(e)

In [9]:
data["sascode"] = data.sasfiles.apply(read_files)

tu_statswithtotal.sas
'utf-8' codec can't decode byte 0xe4 in position 1187: invalid continuation byte


In [10]:
data[data.sasfiles.str.contains('stat')]

,sasfiles,sascode
36,tu_sumstatsinrows.sas,[/*-------------------------------------------...
51,tu_getgstatements.sas,[/*-------------------------------------------...
115,tu_statsfmt.sas,[/*-------------------------------------------...
132,tu_statswithtotal.sas,None
185,tu_stats.sas,[/*-------------------------------------------...
277,tu_sumstatsincols.sas,[/*-------------------------------------------...


In [11]:
question = "How many parameters does BERT-large have?"
answer_text = "BERT-large is really big... it has 24-layers and an embedding size of 1,024, for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take a couple minutes to download to your Colab instance."

In [12]:
!pip install transformers

     |████████████████████████████████| 5.5 MB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 772 kB 89.4 MB/s eta 0:00:01
     |████████████████████████████████| 7.6 MB 89.2 MB/s eta 0:00:01
     |████████████████████████████████| 701 kB 81.2 MB/s eta 0:00:01
     |████████████████████████████████| 163 kB 104.9 MB/s eta 0:00:01


In [13]:
!pip install torch

     |████████████████████████████████| 890.2 MB 3.2 kB/s s eta 0:00:01
     |████████████████████████████████| 849 kB 105.0 MB/s eta 0:00:01
     |████████████████████████████████| 557.1 MB 5.7 kB/s s eta 0:00:01
     |████████████████████████████████| 21.0 MB 105.8 MB/s eta 0:00:01
     |████████████████████████████████| 317.1 MB 14 kB/s /s eta 0:00:01


In [14]:
import torch

In [15]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [16]:
def qanda(question,answer_text):
    input_ids = tokenizer.encode(question, answer_text)
    
    # BERT only needs the token IDs, but for the purpose of inspecting the 
    # tokenizer's behavior, let's also get the token strings and display them.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                                 return_dict=True) 

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):

        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]

        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    return answer

In [17]:
qanda(question,answer_text)

'340m'

In [18]:
dfcode = data[data.sasfiles.str.contains("tc_adpft.sas")]

In [19]:
code = dfcode.sascode.values.tolist()[0]

In [20]:
dfcode.sascode.values.tolist()[0]

['/******************************************************************************* \n',
 '| Program Name: tc_adpft.sas\n',
 '|\n',
 '| Program Version: 1 build 4\n',
 '|\n',
 '| HARP Compound/Study/Reporting Effort: \n',
 '|\n',
 '| Program Purpose: To create the ADaM dataset of Pulmonary Function Tests domain using the SDTM XF\n',
 '|                  and supplemental XF dataset.\n',
 '|\n',
 '| SAS Version: SAS v9.3\n',
 '|\n',
 '| Created By: Robert Croft (rlc25434)\n',
 '| Date:       31/07/2014\n',
 '|\n',
 '|******************************************************************************* \n',
 '|\n',
 '| Output: adamdata.adpft\n',
 '|\n',
 '|\n',
 '|\n',
 '| Nested Macros: \n',
 '| (@) tu_adsuppjoin\n',
 '| (@) tu_addatetime\n',
 '| (@) tu_adgetadslvars\n',
 '| (@) tu_adgettrt\n',
 '| (@) tu_attrib\n',
 '| (@) tu_advisit\n',
 '| (@) tu_adperiod\n',
 '| (@) tu_adreldays\n',
 '| (@) tu_adbaseln\n',
 '| (@) tu_times\n',
 '| (@) tu_decode\n',
 '| (@) tr_putlocals\n',
 '| (@) tu_putglo

In [21]:
dfcode

,sasfiles,sascode
0,tc_adpft.sas,[/********************************************...


In [22]:
codestr = "".join(code)

In [23]:
qanda("Who is the author", codestr[:1000])


'robert croft'

In [24]:
qanda("What is the full purpose", codestr[:1000].replace("|"," "))

'to create the adam dataset of pulmonary function tests domain using the sdtm xf and supplemental xf dataset'

In [25]:
def metadata(codestr):
    try:
        code = codestr
        codestr = "".join(code)
        codestr = codestr.replace("|"," ")
        code = codestr[: 800]

        author = qanda("Who is the creator", code)
        purpose = qanda("What is the full purpose", code)
        version = qanda("What is the SAS version", code)
        Created_Date = qanda("What is the created date", code)
        program_version = qanda("What is the program version and build version if any", code)

        #return author,purpose,version,Created_Date,program_version
        return [author, purpose,version, Created_Date, program_version]
    except:
        print("error")
        return ["","","","",""]

In [26]:
from tqdm import tqdm,tqdm_notebook
tqdm_notebook().pandas()

/tmp/ipykernel_332/2961649409.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


0it [00:00, ?it/s]

In [27]:
data[["author", "purpose","sas version","created date","program version"]] = data.progress_apply(lambda x: metadata(x['sascode']),axis=1,result_type='expand')
#df.apply(lambda x: func(x['col1'],x['col2']),axis=1)

  0%|          | 0/425 [00:00<?, ?it/s]

error


In [28]:
!pip install openpyxl

     |████████████████████████████████| 242 kB 9.1 MB/s eta 0:00:01


In [29]:
 data.to_excel("converted.xlsx")

In [30]:
data = pd.read_excel("converted.xlsx")

In [31]:
print(data)

     Unnamed: 0          sasfiles  \
0             0      tc_adpft.sas   
1             1  tc_p1_vitals.sas   
2             2     td_pkpt3p.sas   
3             3        td_ae1.sas   
4             4      tu_align.sas   
..          ...               ...   
420         420      td_tte11.sas   
421         421     td_liver8.sas   
422         422    tc_p1_elig.sas   
423         423        td_eg9.sas   
424         424     tc_genpro.sas   

                                               sascode  \
0    ['/*******************************************...   
1    ['/*******************************************...   
2    ['/*------------------------------------------...   
3    ['/*\n', '/\n', '/ Macro Name: td_ae1\n', '/\n...   
4    ['/*------------------------------------------...   
..                                                 ...   
420  ['/*******************************************...   
421  ['/*------------------------------------------...   
422  ['/*************************

In [43]:

#data["created date"] = pd.to_datetime(data["created date"])#.dt.strftime("%Y-%m-%d")
#data.to_excel("converted_YYYY_MM_DD.xlsx")
data['Location'] = 'https://mygithub.gsk.com/gsk-tech/standard_mvp/tree/main/Rx/macros/'
data['VAL_STAT'] = 'Production'
data['Type'] = 'Macro'
data['Enviroment'] = 'HARP v10.0.0 - SAS 9.4 - Linux'
data['STD_LEVEL'] = 'Global'
data['SOURCE_LANG'] = 'SAS'
data['DIVISION'] = 'Pharma'
data['CODE_SOURCE'] = 'Internal'
data['OPEN_SOURCE'] = 'NO'
data['Path'] = data['Location'] + data['sasfiles']


ParserError: Unknown string format: o2oct2003

In [47]:
print(data.dtypes)

Unnamed: 0          int64
sasfiles           object
sascode            object
author             object
purpose            object
sas version        object
created date       object
program version    object
Location           object
VAL_STAT           object
Type               object
Enviroment         object
STD_LEVEL          object
SOURCE_LANG        object
DIVISION           object
CODE_SOURCE        object
OPEN_SOURCE        object
Path               object
dtype: object


In [46]:
data['created date'] = data['created date'].astype('str')

In [48]:
print(data.dtypes)

Unnamed: 0          int64
sasfiles           object
sascode            object
author             object
purpose            object
sas version        object
created date       object
program version    object
Location           object
VAL_STAT           object
Type               object
Enviroment         object
STD_LEVEL          object
SOURCE_LANG        object
DIVISION           object
CODE_SOURCE        object
OPEN_SOURCE        object
Path               object
dtype: object


In [33]:
print(data)

     Unnamed: 0          sasfiles  \
0             0      tc_adpft.sas   
1             1  tc_p1_vitals.sas   
2             2     td_pkpt3p.sas   
3             3        td_ae1.sas   
4             4      tu_align.sas   
..          ...               ...   
420         420      td_tte11.sas   
421         421     td_liver8.sas   
422         422    tc_p1_elig.sas   
423         423        td_eg9.sas   
424         424     tc_genpro.sas   

                                               sascode  \
0    ['/*******************************************...   
1    ['/*******************************************...   
2    ['/*------------------------------------------...   
3    ['/*\n', '/\n', '/ Macro Name: td_ae1\n', '/\n...   
4    ['/*------------------------------------------...   
..                                                 ...   
420  ['/*******************************************...   
421  ['/*------------------------------------------...   
422  ['/*************************

In [34]:
data.to_csv("converted.csv", index = False)

In [35]:
!pip install openpyxl

In [39]:
data.to_excel("converted.xlsx")

In [40]:
data

,Unnamed: 0,sasfiles,sascode,author,purpose,sas version,created date,program version,Location,VAL_STAT,Type,Enviroment,STD_LEVEL,SOURCE_LANG,DIVISION,CODE_SOURCE,OPEN_SOURCE,Path
0,0,tc_adpft.sas,['/*******************************************...,robert croft,to create the adam dataset of pulmonary functi...,sas v9 . 3,31 / 07 / 2014,1 build 4,https://mygithub.gsk.com/gsk-tech/standard_mvp...,Production,Macro,HARP v10.0.0 - SAS 9.4 - Linux,Global,SAS,Pharma,Internal,NO,https://mygithub.gsk.com/gsk-tech/standard_mvp...
1,1,tc_p1_vitals.sas,['/*******************************************...,andy miskell,create vitals a & r data set for phase 1 requi...,9 . 1 . 3,"march 18 , 2009",[CLS],https://mygithub.gsk.com/gsk-tech/standard_mvp...,Production,Macro,HARP v10.0.0 - SAS 9.4 - Linux,Global,SAS,Pharma,Internal,NO,https://mygithub.gsk.com/gsk-tech/standard_mvp...
2,2,td_pkpt3p.sas,['/*------------------------------------------...,david sankey / greg weber,this macro shall create the summary statistics...,sas v8,03 - june - 05,[CLS],https://mygithub.gsk.com/gsk-tech/standard_mvp...,Production,Macro,HARP v10.0.0 - SAS 9.4 - Linux,Global,SAS,Pharma,Internal,NO,https://mygithub.gsk.com/gsk-tech/standard_mvp...
3,3,td_ae1.sas,"['/*\n', '/\n', '/ Macro Name: td_ae1\n', '/\n...",john henry king,a macro to create adverse event display,8,29sep2003,[CLS],https://mygithub.gsk.com/gsk-tech/standard_mvp...,Production,Macro,HARP v10.0.0 - SAS 9.4 - Linux,Global,SAS,Pharma,Internal,NO,https://mygithub.gsk.com/gsk-tech/standard_mvp...
4,4,tu_align.sas,['/*------------------------------------------...,yongwei wang,this macro is for optimising column width,sas v9 . 4,16 - june - 03,[SEP],https://mygithub.gsk.com/gsk-tech/standard_mvp...,Production,Macro,HARP v10.0.0 - SAS 9.4 - Linux,Global,SAS,Pharma,Internal,NO,https://mygithub.gsk.com/gsk-tech/standard_mvp...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,420,td_tte11.sas,['/*******************************************...,bob newman ( amadeus ),to generate a tte11 plot,8 . 2,26 october 2006,macro version : 1 build 4 sas version : 8 . 2,https://mygithub.gsk.com/gsk-tech/standard_mvp...,Production,Macro,HARP v10.0.0 - SAS 9.4 - Linux,Global,SAS,Pharma,Internal,NO,https://mygithub.gsk.com/gsk-tech/standard_mvp...
421,421,td_liver8.sas,['/*------------------------------------------...,yongwei wang,display macro to generate idsl liver8,8 . 2,20aug2007,[CLS],https://mygithub.gsk.com/gsk-tech/standard_mvp...,Production,Macro,HARP v10.0.0 - SAS 9.4 - Linux,Global,SAS,Pharma,Internal,NO,https://mygithub.gsk.com/gsk-tech/standard_mvp...
422,422,tc_p1_elig.sas,['/*******************************************...,suzanne johnes,create elig a & r data set for phase1 requirem...,9 . 1,29 april 2008,[CLS],https://mygithub.gsk.com/gsk-tech/standard_mvp...,Production,Macro,HARP v10.0.0 - SAS 9.4 - Linux,Global,SAS,Pharma,Internal,NO,https://mygithub.gsk.com/gsk-tech/standard_mvp...
423,423,td_eg9.sas,['/*******************************************...,bob newman ( amadeus ),to generate an eg9 plot,8 . 2,24 october 2006,macro version : 1 build 3 sas version : 8 . 2,https://mygithub.gsk.com/gsk-tech/standard_mvp...,Production,Macro,HARP v10.0.0 - SAS 9.4 - Linux,Global,SAS,Pharma,Internal,NO,https://mygithub.gsk.com/gsk-tech/standard_mvp...
